In [1]:
library(caret, quiet=TRUE);
library(base64enc)
library(httr)



Attaching package: ‘httr’

The following object is masked from ‘package:caret’:

    progress



# Build a Model

In [2]:
set.seed(1960)

create_model  =  function() {

    model  <- train(Species ~ ., data = iris, method = "glmnet")
    
    return(model)
}

In [3]:
# dataset
model = create_model()

In [4]:
pred <- predict(model, as.matrix(iris[, -5]) , type="prob")
pred_labels <- predict(model, as.matrix(iris[, -5]) , type="raw")
sum(pred_labels != iris$Species)/length(pred_labels)


[1] 0.02666667

# SQL Code Generation

In [5]:

test_ws_sql_gen = function(mod) {
    WS_URL = "https://sklearn2sql.herokuapp.com/model"
    WS_URL = "http://localhost:1888/model"
    model_serialized <- serialize(mod, NULL)
    b64_data = base64encode(model_serialized)
    data = list(Name = "caret_glmnet_test_model", SerializedModel = b64_data , SQLDialect = "postgresql" , Mode="caret")
    r = POST(WS_URL, body = data, encode = "json")
    # print(r)
    content = content(r)
    # print(content)
    lSQL = content$model$SQLGenrationResult[[1]]$SQL # content["model"]["SQLGenrationResult"][0]["SQL"]
    return(lSQL);
}

In [6]:
lModelSQL = test_ws_sql_gen(model)
cat(lModelSQL)

WITH linear_input AS 
(SELECT "ADS"."KEY" AS "KEY", CAST("ADS"."Feature_0" AS FLOAT) AS "Feature_0", CAST("ADS"."Feature_1" AS FLOAT) AS "Feature_1", CAST("ADS"."Feature_2" AS FLOAT) AS "Feature_2", CAST("ADS"."Feature_3" AS FLOAT) AS "Feature_3" 
FROM "INPUT_DATA" AS "ADS"), 
linear_model_cte AS 
(SELECT linear_input."KEY" AS "KEY", -0.9021040991963752 * linear_input."Feature_0" + 2.6454788281043458 * linear_input."Feature_1" + -1.1151551851044692 * linear_input."Feature_2" + -2.2296549826873697 * linear_input."Feature_3" + 5.129773854111001 AS "Score_setosa", 0.09303778049189963 * linear_input."Feature_0" + 0.0 * linear_input."Feature_1" + 0.0 * linear_input."Feature_2" + -0.37346148785070893 * linear_input."Feature_3" + 3.246827263663173 AS "Score_versicolor", 0.0 * linear_input."Feature_0" + -0.9502576420707765 * linear_input."Feature_1" + 1.4979187408924162 * linear_input."Feature_2" + 4.211954278530643 * linear_input."Feature_3" + -8.376601117774172 AS "Score_virginica" 
FROM lin

# Execute the SQL Code

In [7]:
library(RODBC)
conn = odbcConnect("pgsql", uid="db", pwd="db", case="nochange")
odbcSetAutoCommit(conn , autoCommit = TRUE)

[1] 0

In [8]:
dataset = iris[,-5]

df_sql = as.data.frame(dataset)
names(df_sql) = sprintf("Feature_%d",0:(ncol(df_sql)-1))
df_sql$KEY = seq.int(nrow(dataset))

sqlDrop(conn , "INPUT_DATA" , errors = FALSE)
sqlSave(conn, df_sql, tablename = "INPUT_DATA", verbose = FALSE)

head(df_sql)

Feature_0,Feature_1,Feature_2,Feature_3,KEY
5.1,3.5,1.4,0.2,1
4.9,3.0,1.4,0.2,2
4.7,3.2,1.3,0.2,3
4.6,3.1,1.5,0.2,4
5.0,3.6,1.4,0.2,5
5.4,3.9,1.7,0.4,6


In [9]:
# colnames(df_sql)
# odbcGetInfo(conn)
# sqlTables(conn)

In [10]:
df_sql_out = sqlQuery(conn, lModelSQL)
head(df_sql_out)

KEY,Score_setosa,Score_versicolor,Score_virginica,Proba_setosa,Proba_versicolor,Proba_virginica,LogProba_setosa,LogProba_versicolor,LogProba_virginica,Decision,DecisionProba
87,NA,NA,NA,7.724367e-03,0.77543110,0.21684453,-4.863375,-0.25433615,-1.52857462,versicolor,0.7754311
116,NA,NA,NA,7.753639e-05,0.03820586,0.96171661,-9.464763,-3.26476642,-0.03903546,virginica,0.9617166
71,NA,NA,NA,6.484588e-03,0.43973828,0.55377713,-5.038327,-0.82157555,-0.59099296,virginica,0.5537771
68,NA,NA,NA,3.944781e-02,0.94330703,0.01724516,-3.232777,-0.05836346,-4.06022400,versicolor,0.9433070
51,NA,NA,NA,9.925479e-03,0.85621344,0.13386108,-4.612650,-0.15523559,-2.01095273,versicolor,0.8562134
52,NA,NA,NA,1.798519e-02,0.82259683,0.15941798,-4.018207,-0.19528907,-1.83622572,versicolor,0.8225968


# R GLM Output

In [11]:
pred_proba  =  predict(model, as.matrix(iris[,-5]), type = "prob")
df_r_out = data.frame(pred_proba)
names(df_r_out) = sprintf("Proba_%s",model$levels)

df_r_out$KEY = seq.int(nrow(dataset))
df_r_out$Score_setosa  =  NA
df_r_out$Score_versicolor  =  NA
df_r_out$Score_virginica  =  NA
df_r_out$LogProba_setosa  =  log(df_r_out$Proba_setosa)
df_r_out$LogProba_versicolor =  log(df_r_out$Proba_versicolor)
df_r_out$LogProba_virginica  =  log(df_r_out$Proba_virginica)
df_r_out$Decision =   predict(model, as.matrix(iris[,-5]), type = "raw")
df_r_out$DecisionProba =  apply(pred_proba, 1, function(x) max(x))
head(df_r_out)



Proba_setosa,Proba_versicolor,Proba_virginica,KEY,Score_setosa,Score_versicolor,Score_virginica,LogProba_setosa,LogProba_versicolor,LogProba_virginica,Decision,DecisionProba
0.9842407,0.01575925,6.428232e-08,1,NA,NA,NA,-0.01588481,-4.150328,-16.55998,setosa,0.9842407
0.9530563,0.04694337,3.137309e-07,2,NA,NA,NA,-0.04808129,-3.058813,-14.97473,setosa,0.9530563
0.9791711,0.02082876,1.009568e-07,3,NA,NA,NA,-0.02104884,-3.871420,-16.10857,setosa,0.9791711
0.9695958,0.03040395,2.207086e-07,4,NA,NA,NA,-0.03087596,-3.493183,-15.32642,setosa,0.9695958
0.9889968,0.01100320,4.119518e-08,5,NA,NA,NA,-0.01106422,-4.509569,-17.00494,setosa,0.9889968
0.9850546,0.01494520,1.589663e-07,6,NA,NA,NA,-0.01505817,-4.203365,-15.65457,setosa,0.9850546


# Compare R and SQL output

In [13]:
df_merge = merge(x = df_r_out, y = df_sql_out, by = "KEY", all = TRUE, , suffixes = c("_1","_2"))
head(df_merge)

KEY,Proba_setosa_1,Proba_versicolor_1,Proba_virginica_1,Score_setosa_1,Score_versicolor_1,Score_virginica_1,LogProba_setosa_1,LogProba_versicolor_1,LogProba_virginica_1,⋯,Score_versicolor_2,Score_virginica_2,Proba_setosa_2,Proba_versicolor_2,Proba_virginica_2,LogProba_setosa_2,LogProba_versicolor_2,LogProba_virginica_2,Decision_2,DecisionProba_2
1,0.9842407,0.01575925,6.428232e-08,NA,NA,NA,-0.01588481,-4.150328,-16.55998,⋯,NA,NA,0.9842407,0.01575925,6.428232e-08,-0.01588481,-4.150328,-16.55998,setosa,0.9842407
2,0.9530563,0.04694337,3.137309e-07,NA,NA,NA,-0.04808129,-3.058813,-14.97473,⋯,NA,NA,0.9530563,0.04694337,3.137309e-07,-0.04808129,-3.058813,-14.97473,setosa,0.9530563
3,0.9791711,0.02082876,1.009568e-07,NA,NA,NA,-0.02104884,-3.871420,-16.10857,⋯,NA,NA,0.9791711,0.02082876,1.009568e-07,-0.02104884,-3.871420,-16.10857,setosa,0.9791711
4,0.9695958,0.03040395,2.207086e-07,NA,NA,NA,-0.03087596,-3.493183,-15.32642,⋯,NA,NA,0.9695958,0.03040395,2.207086e-07,-0.03087596,-3.493183,-15.32642,setosa,0.9695958
5,0.9889968,0.01100320,4.119518e-08,NA,NA,NA,-0.01106422,-4.509569,-17.00494,⋯,NA,NA,0.9889968,0.01100320,4.119518e-08,-0.01106422,-4.509569,-17.00494,setosa,0.9889968
6,0.9850546,0.01494520,1.589663e-07,NA,NA,NA,-0.01505817,-4.203365,-15.65457,⋯,NA,NA,0.9850546,0.01494520,1.589663e-07,-0.01505817,-4.203365,-15.65457,setosa,0.9850546


In [14]:
diffs_df = df_merge[df_merge$Decision_1 != df_merge$Decision_2,]
head(diffs_df)

Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”

KEY,Proba_setosa_1,Proba_versicolor_1,Proba_virginica_1,Score_setosa_1,Score_versicolor_1,Score_virginica_1,LogProba_setosa_1,LogProba_versicolor_1,LogProba_virginica_1,⋯,Score_versicolor_2,Score_virginica_2,Proba_setosa_2,Proba_versicolor_2,Proba_virginica_2,LogProba_setosa_2,LogProba_versicolor_2,LogProba_virginica_2,Decision_2,DecisionProba_2


In [15]:
stopifnot(nrow(diffs_df) == 0)

In [16]:
summary(df_sql_out)

      KEY         Score_setosa   Score_versicolor Score_virginica
 Min.   :  1.00   Mode:logical   Mode:logical     Mode:logical   
 1st Qu.: 38.25   NA's:150       NA's:150         NA's:150       
 Median : 75.50                                                  
 Mean   : 75.50                                                  
 3rd Qu.:112.75                                                  
 Max.   :150.00                                                  
  Proba_setosa       Proba_versicolor   Proba_virginica    
 Min.   :0.0000000   Min.   :0.002302   Min.   :0.0000000  
 1st Qu.:0.0004788   1st Qu.:0.021167   1st Qu.:0.0000003  
 Median :0.0193688   Median :0.071227   Median :0.0869492  
 Mean   :0.3333383   Mean   :0.333329   Mean   :0.3333331  
 3rd Qu.:0.9656350   3rd Qu.:0.780442   3rd Qu.:0.8182826  
 Max.   :0.9966529   Max.   :0.966136   Max.   :0.9976978  
 LogProba_setosa      LogProba_versicolor LogProba_virginica         Decision 
 Min.   :-16.939128   Min.   :-6.07391 

In [17]:
summary(df_r_out)

  Proba_setosa       Proba_versicolor   Proba_virginica          KEY        
 Min.   :0.0000000   Min.   :0.002302   Min.   :0.0000000   Min.   :  1.00  
 1st Qu.:0.0004788   1st Qu.:0.021167   1st Qu.:0.0000003   1st Qu.: 38.25  
 Median :0.0193688   Median :0.071227   Median :0.0869492   Median : 75.50  
 Mean   :0.3333383   Mean   :0.333329   Mean   :0.3333331   Mean   : 75.50  
 3rd Qu.:0.9656350   3rd Qu.:0.780442   3rd Qu.:0.8182826   3rd Qu.:112.75  
 Max.   :0.9966529   Max.   :0.966136   Max.   :0.9976978   Max.   :150.00  
 Score_setosa   Score_versicolor Score_virginica LogProba_setosa     
 Mode:logical   Mode:logical     Mode:logical    Min.   :-16.939128  
 NA's:150       NA's:150         NA's:150        1st Qu.: -7.658981  
                                                 Median : -3.944091  
                                                 Mean   : -4.532214  
                                                 3rd Qu.: -0.034970  
                                         